In [20]:
from keras.models import load_model
import airsim
import cv2
import numpy as np
import time

In [21]:
model = load_model('model.h5df')

In [22]:
client = airsim.CarClient()
client.confirmConnection()
client.enableApiControl(True)


Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [23]:
def predictionToSteeringAngle(prediction):
    index = np.argmax(prediction)
    if index == 0:
        return -0.5
    elif index == 2:
        return 0.5
    else:
        return 0.0

In [24]:
car_controls = airsim.CarControls()
client.enableApiControl(True)
while True:
    
    response = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.Scene, False, False)])[0]
    img = np.frombuffer(response.image_data_uint8, dtype=np.uint8)
    img = img.reshape(response.height, response.width, 4)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = cv2.resize(img, (128, 128))
    img = np.array(img[None, ..., np.newaxis])
    
    
    prediction = model.predict(img)
    steering_angle = predictionToSteeringAngle(prediction)

    car_controls.throttle = 0.4
    car_controls.steering = steering_angle
    
    client.setCarControls(car_controls)
    
    print("Sending steering {s}".format(s=steering_angle))
    time.sleep(0.1)
    
client.enableApiControl(False)

Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steering 0.5
Sending steer

KeyboardInterrupt: 